In [3]:
# Set env variable before import of vds_access!
import os
os.environ["WEBVIZ_VDS_HOST_ADDRESS"] = "https://server-oneseismictest-dev.playground.radix.equinor.com"

from sumo.wrapper import SumoClient


# from fmu.sumo.explorer import Explorer
# from fmu.sumo.explorer.objects.cube_collection import CubeCollection
from fmu.sumo.explorer.objects import CaseCollection

from services.vds_access.vds_access import VdsAccess
# from services.vds_access. import VdsMetadata
from services.sumo_access.seismic_access import SeismicAccess
from services.vds_access.request_types import VdsCoordinates
from services.vds_access.response_types import VdsMetadata

from src.services.sumo_access._helpers import SumoCase


import numpy as np



# Test data
case_uuid = "c619f32d-3ada-4e5e-8d3c-330f940e88f8"
realization = 0
iteration = "iter-0"
cube_tagname = "amplitude_far_time"
timestep = None
timestamp = "2018-01-01T00:00:00"
observed = False

sumo_client = SumoClient(env="prod")
case_collection = CaseCollection(sumo_client).filter(uuid=case_uuid) # Existing working case
case = case_collection[0]

sumo_case = await SumoCase.from_case_uuid(sumo_client.authenticate(), case_uuid=case_uuid)

# Setup
seismic_access = SeismicAccess(sumo_client=sumo_client, case=case, case_uuid=case_uuid, iteration_name=iteration)

vds_handle = await seismic_access.get_vds_handle(seismic_attribute=cube_tagname,realization=realization, time_or_interval_str=timestamp)
vds_access = VdsAccess(sas_token=vds_handle.sas_token, vds_url=vds_handle.vds_url)



x_points = [463156.911, 463564.402, 463637.925, 463690.658, 463910.452]
y_points = [5929542.294, 5931057.803, 5931184.235, 5931278.837, 5931688.122]

x_points = [463156.911, 463564.402, 463637.925, 463690.658]
y_points = [5929542.294, 5931057.803, 5931184.235, 5931278.837,]

[
    flattened_fence_traces_array,
    num_traces,
    num_trace_samples,
]  = await vds_access.get_flattened_fence_traces_array_and_metadata(coordinates=VdsCoordinates(x_points=x_points, y_points=y_points))


meta: VdsMetadata = await vds_access.get_metadata()
if len(meta.axis) != 3:
    raise ValueError(f"Expected 3 axes, got {len(meta.axis)}")
depth_axis_meta = meta.axis[2]
print(meta)
print(f'Vds Access coordinate system: {meta.crs}')

# coordinate_system = await sumo_case.get_coordinate_system_identifier()
# print(f'Sumo case coordinate system: {coordinate_system}')

print(f'Flattened fence traces array: {flattened_fence_traces_array}')
print(f'Number of traces: {num_traces}')
print(f'Number of samples in trace: {num_trace_samples}')


# Test code:
test_array = [["t11","t12","t13"], ["t21","t22","t23"], ["t31","t32","t33"]]
test_flattened_array_C = np.asarray(test_array).ravel(order='C')
test_flattened_array_F = np.asarray(test_array).ravel(order='F')
display(test_array)
display(test_flattened_array_C)
display(test_flattened_array_F)




axis=[VdsAxis(annotation='Inline', max=217.0, min=0.0, samples=218, unit='unitless'), VdsAxis(annotation='Crossline', max=137.0, min=0.0, samples=138, unit='unitless'), VdsAxis(annotation='Sample', max=1999.0, min=1500.0, samples=500, unit='ms')] boundingBox=VdsBoundingBox(cdp=[[461484.34, 5926563.2], [456083.37, 5936006.48], [462069.5080733945, 5939430.173027524], [467470.4780733945, 5929986.8930275235]], ij=[[0.0, 0.0], [217.0, 0.0], [217.0, 137.0], [0.0, 137.0]], ilxl=[[0.0, 0.0], [217.0, 0.0], [217.0, 137.0], [0.0, 137.0]]) crs=''
Vds Access coordinate system: 
Sumo case coordinate system: ST_WGS84_UTM37N_P32637
Flattened fence traces array: [0. 0. 0. ... 0. 0. 0.]
Number of traces: 4
Number of samples in trace: 500


[['t11', 't12', 't13'], ['t21', 't22', 't23'], ['t31', 't32', 't33']]

array(['t11', 't12', 't13', 't21', 't22', 't23', 't31', 't32', 't33'],
      dtype='<U3')

array(['t11', 't21', 't31', 't12', 't22', 't32', 't13', 't23', 't33'],
      dtype='<U3')